In [32]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [33]:
df_fake=pd.read_csv("Fake.csv")
df_real=pd.read_csv("True.csv")
df_fake["class"]=0
df_real["class"]=1

df_merge=pd.concat([df_fake,df_real],axis=0)
df=df_merge.drop(["title","subject","date"],axis=1)
df=df.sample(frac=1).reset_index(drop=True)
df

,text,class
0,Former President George H. W. Bush can teach D...,0
1,WASHINGTON (Reuters) - U.S. Defense Secretary ...,1
2,WASHINGTON (Reuters) - Donald Trump’s new camp...,1
3,Republican Presidential front runner Donald Tr...,0
4,WASHINGTON (Reuters) - The Trump administratio...,1
...,...,...
44893,"At a Wisconsin high school soccer game, player...",0
44894,Like the Soldiers of Oden vigilante group we r...,0
44895,PARIS (Reuters) - France will on Thursday anno...,1
44896,WASHINGTON (Reuters) - In a heated moment duri...,1


In [34]:
df["class"]=df["class"].replace({0:"fake",1:"real"})
df

,text,class
0,Former President George H. W. Bush can teach D...,fake
1,WASHINGTON (Reuters) - U.S. Defense Secretary ...,real
2,WASHINGTON (Reuters) - Donald Trump’s new camp...,real
3,Republican Presidential front runner Donald Tr...,fake
4,WASHINGTON (Reuters) - The Trump administratio...,real
...,...,...
44893,"At a Wisconsin high school soccer game, player...",fake
44894,Like the Soldiers of Oden vigilante group we r...,fake
44895,PARIS (Reuters) - France will on Thursday anno...,real
44896,WASHINGTON (Reuters) - In a heated moment duri...,real


In [35]:
patterns = {
            r'\d+': '',             # remove digits (numeri)
            r'[^\w\s]': '',         # remove punteggiatura e simboli ...,'@!£$%
            r'\b\w{1,2}\b':'',      # remove all token less than2 characters
            r'(http|www)[^\s]+':'', # remove website
            r'\s+': ' '             # sostituisce tutti i multipli spazi con uno spazio
            }

def clean_column(df, column, patterns):
    for pattern, replacement in patterns.items():
        df[column] = df[column].str.replace(pattern, replacement)
        df[column] = df[column].str.lower() # applica il lower
    return df

#solo in preview
clean_column(df, 'text', patterns)
df

,text,class
0,former president george bush can teach donald ...,fake
1,washington reuters defense secretary ash carte...,real
2,washington reuters donald trumps new campaign ...,real
3,republican presidential front runner donald tr...,fake
4,washington reuters the trump administration mu...,real
...,...,...
44893,wisconsin high school soccer game players fro...,fake
44894,like the soldiers oden vigilante group reporte...,fake
44895,paris reuters france will thursday announce pl...,real
44896,washington reuters heated moment during his un...,real


In [36]:
df.isnull().sum()


text     0
class    0
dtype: int64

In [37]:
fake=df[df["class"]=="fake"]["text"]
real=df[df["class"]=="real"]["text"]



In [38]:
textfake = list(map(str, fake))
textfake = ''.join(textfake)

In [39]:
textreal = list(map(str, real))
textreal = ''.join(textreal)

In [40]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

wordcloud = WordCloud(background_color = 'black', width = 800, height = 400,
                      max_words = 180, contour_width = 3,
                      max_font_size = 80, contour_color = 'steelblue',
                      stopwords = STOPWORDS, random_state = 667)

wordcloud.generate(textreal)

plt.figure(figsize=(18,10))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis("off")

ValueError: Only supported for TrueType fonts

In [41]:
X=df["text"]
y=df["class"]

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7)
#vectorizer = CountVectorizer(stop_words='english')
X_vect = vectorizer.fit_transform(X).toarray()
X_vect

array([[0, 0, 1, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 4, 1, 0],
       [0, 0, 0, ..., 1, 0, 0]], dtype=int64)

In [43]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidfconverter = TfidfTransformer()
X_tfidf = tfidfconverter.fit_transform(X_vect).toarray()
X_tfidf

array([[0.        , 0.        , 0.04132739, ..., 0.        , 0.        ,
        0.03664561],
       [0.        , 0.        , 0.        , ..., 0.09364658, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.08500144, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.13321176, 0.06172626,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.14473379, 0.        ,
        0.        ]])

In [44]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y,
                                                    test_size=0.3, 
                                                    random_state=667
                                                    )

In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
classifier = RandomForestClassifier(n_estimators=1000, random_state=667,max_depth=5)
classifier.fit(X_train, y_train) 

# accuracy score on the test data
y_pred = classifier.predict(X_test)
test_data_accuracy = accuracy_score(y_pred, y_test)
test_data_accuracy

0.9848552338530067

In [ ]:
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.metrics import accuracy_score

#classifier = MultinomialNB(alpha=0.1)
#classifier.fit(X_train, y_train) 

# accuracy score on the test data
#y_pred = classifier.predict(X_test)
#test_data_accuracy = accuracy_score(y_pred, y_test)
#test_data_accuracy

0.9308092056421677

In [46]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        fake       0.99      0.98      0.99      6966
        real       0.98      0.99      0.98      6504

    accuracy                           0.98     13470
   macro avg       0.98      0.98      0.98     13470
weighted avg       0.98      0.98      0.98     13470



In [51]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix 

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize = (10,8))
sns.heatmap(cm,cmap= "Blues", 
            linecolor = 'black', 
            linewidth = 1, 
            annot = True, 
            fmt='', 
            xticklabels = classifier.classes_, 
            yticklabels = classifier.classes_)

plt.xlabel("Predicted")
plt.ylabel("Actual")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\ifoa\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
  File "C:\Users\ifoa\AppData\Local\Temp\ipykernel_14652\2392853767.py", line 6, in <module>
    plt.figure(figsize = (10,8))
  File "c:\Users\ifoa\AppData\Local\Programs\Python\Python39\lib\site-packages\matplotlib\pyplot.py", line 808, in figure
  File "c:\Users\ifoa\AppData\Local\Programs\Python\Python39\lib\site-packages\matplotlib\pyplot.py", line 326, in new_figure_manager
    manager_class = getattr(getattr(backend_mod, "FigureCanvas", None),
  File "c:\Users\ifoa\AppData\Local\Programs\Python\Python39\lib\site-packages\matplotlib\pyplot.py", line 316, in _warn_if_gui_out_of_main_thread
    if manager:
  File "c:\Users\ifoa\AppData\Local\Programs\Python\Python39\lib\site-packages\matplotlib\pyplot.py", line 217, in _get_backend_mod
    another interactive backend has started.  Switching to and from
  File "c:\Users\ifoa\

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3, 
                                                    random_state=667
                                                    )


from sklearn.pipeline import Pipeline

bow = CountVectorizer(max_features=1500, min_df=5, max_df=0.7)
tfidf = TfidfTransformer()
clf = RandomForestClassifier(n_estimators=1000, random_state=667,max_depth=5)

pipe = Pipeline([
                ('bow',bow),
                ('tfidf',tfidf),
                ('clf',clf),
                ])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

test_data_accuracy = accuracy_score(y_test,y_pred)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9850779510022272


In [53]:
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(pipe, X, y, scoring = 'f1_micro', cv = 8)

print(f'scores={scores}')
print(f'mean={np.mean(scores)}')
print(f'std={np.std(scores)}')

scores=[0.98574737 0.98450027 0.98770492 0.98574483 0.98574483 0.98235923
 0.98788311 0.98325018]
mean=0.9853668423102413
std=0.0018202230360848308


In [54]:
import joblib
joblib.dump(pipe,'NLPEs1.pkl')

['NLPEs1.pkl']

In [55]:

uploaded_model = joblib.load('NLPEs1.pkl')
pred = uploaded_model.predict([df["text"][3]]) 
pred[0]

'fake'

In [60]:
pred = uploaded_model.predict(["washington reuters defense secretary ash"]) 
pred[0]

'real'